# Azure AI Agent Service - AI Search

### *Create Azure AI Search agent to search over the documents indexed using Azure AI search service*

<span style="color:red"> Please refer *pre-requisite* section in readme file before running this notebook! </span>

<span style="color:red"> This notebook is created to work based on Foundry hub based project set up, These instructions won't work on new Foundry projects (v2)! </span>

#### Step 1

* Rename *credentials.txt* file to *credentials.env* file 

* Add all required configuration values in the *credentials.env* file

* Select kernel as *Python 3* in the top right corner of the notebook (Python version 3.12)



#### Step 2

* Login to Azure account, by following the below steps:

    * Right click on this notebook from left menu and open in terminal
    
    * Run the command `az login` and follow the instructions to login to your Azure account
    
    (If you see any issues logging in to Azure CLI, error might be due to azure policies, try different tenant/subscription)

In [ ]:
import os
from datetime import datetime as pydatetime
from typing import Any, List, Dict
from dotenv import load_dotenv
# Azure AI Projects
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import ConnectionType
from azure.ai.projects.models import AzureAISearchTool, AzureAISearchQueryType

In [2]:
# Step 3 (Optional) - Not required if you are using codespaces or Jupyter notebooks with pre-installed dependencies.

# If you encounter any errors during execution of above cell, install dependencies manually by running below command:
# Make sure you are using python version > 3.10

# %pip install -r requirements.txt

Load variables from env file

In [ ]:
# load secrets
load_dotenv("credentials.env", override=True)
aiConnectionStr=os.environ["AIPROJECT_CONNECTION_STRING"]
modelName=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")
aiSearchConnectionName = os.getenv("AISEARCH_CONNECTION_NAME")
indexName = os.getenv("AISEARCH_INDEX_NAME")

Check version of AI projects library. Make sure you are using version 1.0.0b10

In [ ]:
import azure.ai.projects
print(azure.ai.projects.__version__)

In [ ]:
# Create Client and Load Azure AI Foundry project client
credential = DefaultAzureCredential(exclude_interactive_browser_credential=False)
project_client = AIProjectClient.from_connection_string(
    credential=credential,
    scope=['https://management.azure.com','https://management.core.windows.net','https://management.core.windows.net/','https://management.core.windows.net','https://management.azure.com/','https://management.azure.com'],
    conn_str=aiConnectionStr,   
)

In [ ]:
# List all agents to see what's already available
all_agents = project_client.agents.list_agents().data
print(f"Found {len(all_agents)} agents:")
for agent in all_agents:
	print(f"- {agent.name} (ID: {agent.id})")

In [ ]:
#AI search connection
aisearch_connection = project_client.connections.get(
    connection_name=aiSearchConnectionName,    
    include_credentials=True,  # Optional. Defaults to "False".
)
print(aisearch_connection)

------

ex-1: Enter input prompt to search over the documents indexed using Azure AI search service 
and name the agent as *AISearchWorkshopAgent*

In [ ]:
newAgentName = "AISearchWorkshopAgent"
input_prompt = "does perks plus cover gym membership?"

ex-2: Create new AI search agent

In [ ]:

# Initialize agent AI search tool and add the search index connection id
ai_search = AzureAISearchTool(
    index_connection_id=aisearch_connection.id, index_name=indexName, query_type=AzureAISearchQueryType.SEMANTIC, top_k=3, filter=""
)

# Create agent with AI search tool and process assistant run

agent = project_client.agents.create_agent(
    model=modelName,
    name=newAgentName,
    instructions="You are a helpful assistant",
    tools=ai_search.definitions,
    tool_resources=ai_search.resources
)

ex-3: Create thread and add messages to the thread

In [ ]:
# Create thread for communication
thread = project_client.agents.create_thread()
print(f"Created thread, ID: {thread.id}")

# Create message to thread
# Remember to update the message with your data
message = project_client.agents.create_message(
    thread_id=thread.id,
    role="user",
    content=input_prompt,
)
print(f"Created message, ID: {message.id}")

ex-4: Run the agent and get the response

In [ ]:
# Create and process agent run in thread with tools
run = project_client.agents.create_and_process_run(thread_id=thread.id, agent_id=agent.id)
print(f"Run finished with status: {run.status}")

if run.status == "failed":
    print(f"Run failed: {run.last_error}")

# Fetch and log all messages
messages = project_client.agents.list_messages(thread_id=thread.id)
print(f"Messages: {messages}")


----

Response recvd: 

In [ ]:
agent_response = messages.data[0].content[0].text.value# Get the content of the first message]
print(agent_response)

Clean up

In [ ]:
#Delete the assistant when done
project_client.agents.delete_agent(agent.id)
print("Deleted agent")
project_client.agents.delete_thread(thread.id)
print("Deleted thread")